# Polymarket Trading API

## Table of Contents

**No Side Effects (Read-Only):**
1. API Connection & Credentials Check
2. Balance Check
3. List Markets
7. Search Markets
8. Portfolio & PnL
9. Orderbook Visualization
10. Spread Detection (Arbitrage)
13. Market Details

**With Side Effects (Modify State):**
4. Place Order
5. Check Updated Balance
6. Cancel Orders
11. Limit Order
12. Export Trade History

## 13. Market Details

In [ ]:
# Export trades to CSV
trades = client.get_trades()

if trades:
    df_trades = pd.DataFrame([{
        'timestamp': trade.get('timestamp'),
        'side': trade.get('side'),
        'size': trade.get('size'),
        'price': trade.get('price'),
        'asset_id': trade.get('asset_id')[:30] + '...',
        'fee': trade.get('fee', 0),
    } for trade in trades])
    
    # Save to CSV
    filename = f"polymarket_trades_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_trades.to_csv(filename, index=False)
    print(f"✅ Exported {len(trades)} trades to {filename}")
    
    display(df_trades)
else:
    print("No trades to export")

## 12. Export Trade History

In [ ]:
# Place limit order at custom price (won't execute immediately)
print("=" * 50)
print("LIMIT ORDER")
print("=" * 50)

if token_id:
    # Show current orderbook
    ob = client.get_order_book(token_id)
    if ob.bids and ob.asks:
        print(f"Best Bid: ${float(ob.bids[0].price):.4f}")
        print(f"Best Ask: ${float(ob.asks[0].price):.4f}")
    
    # User inputs
    side = input("\nSide (BUY/SELL): ").strip().upper() or "BUY"
    price = float(input("Limit Price: ") or "0.50")
    size = float(input("Size (shares): ") or "10")
    
    total = price * size
    print(f"\nOrder Summary:")
    print(f"  {side} {size} shares @ ${price:.4f} = ${total:.2f}")
    
    confirm = input("\nPlace limit order? (yes/no): ").strip().lower()
    
    if confirm == "yes":
        try:
            order_args = OrderArgs(
                price=price,
                size=size,
                side=side,
                token_id=token_id,
            )
            
            signed_order = client.create_order(order_args)
            result = client.post_order(signed_order, OrderType.GTC)  # Good Till Cancelled
            
            print(f"\n✅ Limit order placed!")
            print(f"Order ID: {result.get('orderID', 'N/A')}")
            print(f"Status: {result.get('status', 'N/A')}")
        except Exception as e:
            print(f"\n❌ Order failed: {e}")
    else:
        print("\nOrder cancelled.")
else:
    print("No token selected")

## 11. Limit Order (Custom Price)

In [ ]:
# Detect spread opportunities (YES + NO < $1.00 = profit)
# Using concurrent requests for speed
import concurrent.futures
from time import time

print("Scanning for spread opportunities (parallel)...\n")

start_time = time()

resp = requests.get(f"{HOST}/markets?limit=500")
data = resp.json()
markets = data.get("data", []) if isinstance(data, dict) else data

# Filter markets with 2 tokens
valid_markets = [m for m in markets if len(m.get("tokens", [])) >= 2]
print(f"Markets to scan: {len(valid_markets)}")

def check_spread(market):
    """Check spread for a single market"""
    try:
        tokens = market.get("tokens", [])
        yes_token = tokens[0].get("token_id")
        no_token = tokens[1].get("token_id")
        
        yes_ob = client.get_order_book(yes_token)
        no_ob = client.get_order_book(no_token)
        
        if not yes_ob.asks or not no_ob.asks:
            return None
        
        yes_ask = float(yes_ob.asks[0].price)
        no_ask = float(no_ob.asks[0].price)
        spread = yes_ask + no_ask
        
        if spread < 1.0:
            profit_pct = (1.0 - spread) * 100
            return {
                'question': market.get('question', '')[:50],
                'yes_ask': yes_ask,
                'no_ask': no_ask,
                'spread': spread,
                'profit_pct': profit_pct,
                'yes_token': yes_token,
                'no_token': no_token,
            }
    except:
        pass
    return None

# Parallel scan with ThreadPoolExecutor
opportunities = []
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    results = executor.map(check_spread, valid_markets[:200])
    opportunities = [r for r in results if r is not None]

elapsed = time() - start_time

# Sort by profit
opportunities.sort(key=lambda x: x['profit_pct'], reverse=True)

print(f"Scanned {min(200, len(valid_markets))} markets in {elapsed:.2f}s")
print(f"Found {len(opportunities)} opportunities:\n")
print("=" * 70)

for opp in opportunities[:10]:
    print(f"Market: {opp['question']}...")
    print(f"  YES: ${opp['yes_ask']:.4f}  NO: ${opp['no_ask']:.4f}")
    print(f"  Spread: ${opp['spread']:.4f}  Profit: {opp['profit_pct']:.2f}%")
    print()

## 10. Spread Detection (Arbitrage)

In [ ]:
# Visualize orderbook depth
import matplotlib.pyplot as plt

if token_id:
    ob = client.get_order_book(token_id)
    
    # Prepare data
    bid_prices = [float(b.price) for b in ob.bids] if ob.bids else []
    bid_sizes = [float(b.size) for b in ob.bids] if ob.bids else []
    ask_prices = [float(a.price) for a in ob.asks] if ob.asks else []
    ask_sizes = [float(a.size) for a in ob.asks] if ob.asks else []
    
    # Cumulative depth
    bid_cumsum = []
    ask_cumsum = []
    cum = 0
    for s in bid_sizes:
        cum += s
        bid_cumsum.append(cum)
    cum = 0
    for s in ask_sizes:
        cum += s
        ask_cumsum.append(cum)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart
    ax1.barh(range(len(bid_prices)), bid_sizes, color='green', alpha=0.7, label='Bids')
    ax1.barh(range(len(ask_prices)), [-s for s in ask_sizes], color='red', alpha=0.7, label='Asks')
    ax1.set_xlabel('Size')
    ax1.set_ylabel('Price Level')
    ax1.set_title('Order Book Depth')
    ax1.legend()
    ax1.axvline(x=0, color='black', linewidth=0.5)
    
    # Depth chart
    if bid_prices and ask_prices:
        ax2.fill_between(bid_prices, bid_cumsum, alpha=0.5, color='green', label='Bid Depth')
        ax2.fill_between(ask_prices, ask_cumsum, alpha=0.5, color='red', label='Ask Depth')
        ax2.set_xlabel('Price')
        ax2.set_ylabel('Cumulative Size')
        ax2.set_title('Market Depth')
        ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Stats
    if bid_prices and ask_prices:
        spread = ask_prices[0] - bid_prices[0]
        mid = (ask_prices[0] + bid_prices[0]) / 2
        print(f"Best Bid: ${bid_prices[0]:.4f} (size: {bid_sizes[0]:.2f})")
        print(f"Best Ask: ${ask_prices[0]:.4f} (size: {ask_sizes[0]:.2f})")
        print(f"Spread:   ${spread:.4f} ({spread/mid*100:.2f}%)")
        print(f"Mid:      ${mid:.4f}")
else:
    print("No token selected")

## 9. Orderbook Visualization

In [ ]:
# Calculate portfolio from trade history
trades = client.get_trades()

if trades:
    # Group trades by asset
    positions = {}
    
    for trade in trades:
        asset_id = trade.get('asset_id')
        side = trade.get('side')
        size = float(trade.get('size', 0))
        price = float(trade.get('price', 0))
        
        if asset_id not in positions:
            positions[asset_id] = {
                'quantity': 0,
                'total_cost': 0,
                'trades': 0
            }
        
        if side == 'BUY':
            positions[asset_id]['quantity'] += size
            positions[asset_id]['total_cost'] += size * price
        else:  # SELL
            positions[asset_id]['quantity'] -= size
            positions[asset_id]['total_cost'] -= size * price
        
        positions[asset_id]['trades'] += 1
    
    print("=" * 60)
    print("PORTFOLIO POSITIONS")
    print("=" * 60)
    
    total_value = 0
    total_cost = 0
    
    for asset_id, pos in positions.items():
        if pos['quantity'] <= 0:
            continue
            
        # Get current price
        try:
            ob = client.get_order_book(asset_id)
            if ob.bids:
                current_price = float(ob.bids[0].price)
            elif ob.asks:
                current_price = float(ob.asks[0].price)
            else:
                current_price = 0.5  # Default
        except:
            current_price = 0.5
        
        avg_cost = pos['total_cost'] / pos['quantity'] if pos['quantity'] > 0 else 0
        current_value = pos['quantity'] * current_price
        pnl = current_value - pos['total_cost']
        pnl_pct = (pnl / pos['total_cost'] * 100) if pos['total_cost'] > 0 else 0
        
        total_value += current_value
        total_cost += pos['total_cost']
        
        print(f"\nToken: {asset_id[:25]}...")
        print(f"  Quantity: {pos['quantity']:.2f}")
        print(f"  Avg Cost: ${avg_cost:.4f}")
        print(f"  Current:  ${current_price:.4f}")
        print(f"  Value:    ${current_value:.2f}")
        print(f"  PnL:      ${pnl:+.2f} ({pnl_pct:+.1f}%)")
    
    print("\n" + "=" * 60)
    total_pnl = total_value - total_cost
    print(f"TOTAL VALUE: ${total_value:.2f}")
    print(f"TOTAL COST:  ${total_cost:.2f}")
    print(f"TOTAL PnL:   ${total_pnl:+.2f}")
    print("=" * 60)
else:
    print("No trades found")

## 8. Portfolio & PnL

In [ ]:
# Search markets by keyword
keyword = input("Search keyword (e.g., bitcoin, trump, election): ").strip().lower()

if keyword:
    # Fetch all markets
    resp = requests.get(f"{HOST}/markets?limit=500")
    data = resp.json()
    all_markets = data.get("data", []) if isinstance(data, dict) else data
    
    # Filter by keyword
    matching = [m for m in all_markets if keyword in m.get("question", "").lower()]
    
    print(f"\nFound {len(matching)} markets matching '{keyword}':\n")
    
    for i, m in enumerate(matching[:15]):
        question = m.get("question", "")[:60]
        active = "✅" if m.get("active") else "❌"
        accepting = "📈" if m.get("accepting_orders") else "⏸️"
        print(f"{i+1}. {active}{accepting} {question}...")
else:
    print("No keyword entered")

## 7. Search Markets

In [ ]:
# Cancel ALL open orders
if orders:
    confirm = input(f"Cancel ALL {len(orders)} orders? (yes/no): ").strip().lower()
    
    if confirm == "yes":
        try:
            result = client.cancel_all()
            print(f"✅ All orders cancelled!")
        except Exception as e:
            print(f"❌ Cancel all failed: {e}")
    else:
        print("Skipped")

In [ ]:
# Cancel a specific order
if orders:
    order_num = int(input("Enter order number to cancel (0 to skip): ") or "0")
    
    if order_num > 0 and order_num <= len(orders):
        order_to_cancel = orders[order_num - 1]
        order_id = order_to_cancel.get('id')
        
        try:
            result = client.cancel(order_id)
            print(f"✅ Order cancelled: {order_id[:30]}...")
        except Exception as e:
            print(f"❌ Cancel failed: {e}")
    else:
        print("Skipped")
else:
    print("No orders to cancel")

In [ ]:
# View open orders
orders = client.get_orders()
print(f"Open Orders: {len(orders)}\n")

if orders:
    for i, order in enumerate(orders):
        print(f"{i+1}. {order.get('side')} {order.get('size')} @ ${order.get('price')}")
        print(f"   Order ID: {order.get('id', 'N/A')[:30]}...")
        print()
else:
    print("No open orders")

# Polymarket Trading API

This notebook covers:
1. API Connection & Credentials Check
2. Balance Check
3. List Markets
4. Place Order

## Setup

In [ ]:
import os
import hmac
import hashlib
import base64
import requests
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / ".env")

from py_clob_client.client import ClobClient
from py_clob_client.clob_types import ApiCreds, OrderArgs, OrderType, BalanceAllowanceParams, AssetType

HOST = "https://clob.polymarket.com"
CHAIN_ID = 137

## 1. API Connection & Credentials Check

In [ ]:
# Load credentials from .env
PRIVATE_KEY = os.getenv("WALLET_PRIVATE_KEY", "")
PROXY_WALLET = os.getenv("POLYMARKET_WALLET_ADDRESS", "")

if not PRIVATE_KEY:
    raise ValueError("Missing WALLET_PRIVATE_KEY in .env")

if not PRIVATE_KEY.startswith("0x"):
    PRIVATE_KEY = "0x" + PRIVATE_KEY

print(f"Private Key: {PRIVATE_KEY[:6]}...{PRIVATE_KEY[-4:]}")
print(f"Proxy Wallet: {PROXY_WALLET}")

In [ ]:
# Initialize client with proxy wallet support
client = ClobClient(
    HOST,
    key=PRIVATE_KEY,
    chain_id=CHAIN_ID,
    signature_type=2,  # POLY_GNOSIS_SAFE (proxy wallet)
    funder=PROXY_WALLET if PROXY_WALLET else None,
)

print(f"Signer Address: {client.signer.address()}")

# Derive and set API credentials
creds = client.create_or_derive_api_creds()
client.set_api_creds(creds)

print(f"API Key: {creds.api_key[:8]}...{creds.api_key[-4:]}")
print("\n✅ Client initialized successfully!")

In [ ]:
# Test API connection
server_time = client.get_server_time()
print(f"Server Time: {server_time}")
print(f"Local Time:  {int(datetime.now().timestamp())}")
print("\n✅ API connection working!")

## 2. Balance Check

In [ ]:
# Check USDC balance
balance_params = BalanceAllowanceParams(
    asset_type=AssetType.COLLATERAL,
    signature_type=2  # Use proxy wallet
)
balance_data = client.get_balance_allowance(balance_params)

balance_raw = int(balance_data.get("balance", 0))
balance_usdc = balance_raw / 1_000_000  # 6 decimals

print(f"Balance (raw): {balance_raw}")
print(f"Balance (USDC): ${balance_usdc:.2f}")
print(f"\nAllowances: {balance_data.get('allowances', {})}")

In [ ]:
# Check open orders
orders = client.get_orders()
print(f"Open Orders: {len(orders)}")

for order in orders[:5]:
    print(f"  - {order.get('side')} {order.get('size')} @ ${order.get('price')}")

In [ ]:
# Check trade history
trades = client.get_trades()
print(f"Trade History: {len(trades)} trades")

for trade in trades[:5]:
    side = trade.get('side')
    size = trade.get('size')
    price = trade.get('price')
    print(f"  - {side} {size} @ ${price}")

## 3. List Markets

In [ ]:
# Fetch markets from API
resp = requests.get(f"{HOST}/markets?limit=100")
data = resp.json()
markets = data.get("data", []) if isinstance(data, dict) else data

print(f"Total markets fetched: {len(markets)}")

# Filter active markets
active_markets = [m for m in markets if m.get("active") and m.get("accepting_orders")]
print(f"Active markets (accepting orders): {len(active_markets)}")

In [ ]:
# Display active markets
import pandas as pd

if active_markets:
    df = pd.DataFrame([{
        'Question': m.get('question', '')[:60],
        'Active': m.get('active'),
        'Accepting': m.get('accepting_orders'),
        'Tokens': len(m.get('tokens', [])),
    } for m in active_markets[:20]])
    display(df)
else:
    print("No active markets accepting orders")

In [ ]:
# Find markets with active orderbooks
markets_with_orderbooks = []

for market in markets[:50]:
    tokens = market.get("tokens", [])
    if not tokens:
        continue
    
    token_id = tokens[0].get("token_id")
    try:
        ob = client.get_order_book(token_id)
        if ob.asks or ob.bids:
            markets_with_orderbooks.append({
                'question': market.get('question', '')[:50],
                'token_id': token_id,
                'bids': len(ob.bids) if ob.bids else 0,
                'asks': len(ob.asks) if ob.asks else 0,
                'best_bid': float(ob.bids[0].price) if ob.bids else None,
                'best_ask': float(ob.asks[0].price) if ob.asks else None,
            })
            if len(markets_with_orderbooks) >= 10:
                break
    except:
        continue

print(f"Markets with active orderbooks: {len(markets_with_orderbooks)}")

if markets_with_orderbooks:
    df = pd.DataFrame(markets_with_orderbooks)
    display(df)

## 4. Place Order

In [ ]:
# Select a market to trade
# Option 1: Use token from recent trade
# Option 2: Use token from markets_with_orderbooks

if trades:
    token_id = trades[0].get('asset_id')
    print(f"Using token from recent trade: {token_id[:30]}...")
elif markets_with_orderbooks:
    token_id = markets_with_orderbooks[0]['token_id']
    print(f"Using token from market: {markets_with_orderbooks[0]['question']}")
else:
    token_id = None
    print("No active market found!")

In [ ]:
# Get orderbook for selected token
if token_id:
    orderbook = client.get_order_book(token_id)
    
    print("=" * 50)
    print("ORDER BOOK")
    print("=" * 50)
    
    print("\nAsks (Sell orders):")
    if orderbook.asks:
        for ask in orderbook.asks[:5]:
            print(f"  ${float(ask.price):.2f} - Size: {float(ask.size):.2f}")
    else:
        print("  No asks")
    
    print("\nBids (Buy orders):")
    if orderbook.bids:
        for bid in orderbook.bids[:5]:
            print(f"  ${float(bid.price):.2f} - Size: {float(bid.size):.2f}")
    else:
        print("  No bids")
    
    print("\n" + "=" * 50)
    if orderbook.asks and orderbook.bids:
        spread = float(orderbook.asks[0].price) - float(orderbook.bids[0].price)
        print(f"Spread: ${spread:.4f}")

In [ ]:
# User input for order
print("=" * 50)
print("PLACE ORDER")
print("=" * 50)

side = input("Side (BUY/SELL): ").strip().upper() or "BUY"
amount_usd = float(input("Amount in USD (e.g., 1.0): ") or "1.0")

# Get current price
if side == "BUY" and orderbook.asks:
    price = float(orderbook.asks[0].price)
    print(f"\nUsing best ask price: ${price:.2f}")
elif side == "SELL" and orderbook.bids:
    price = float(orderbook.bids[0].price)
    print(f"\nUsing best bid price: ${price:.2f}")
else:
    price = float(input("Price: "))

# Calculate size
import math
size = math.ceil(amount_usd / price * 100) / 100  # Round up
actual_cost = size * price

print(f"\nOrder Summary:")
print(f"  Side: {side}")
print(f"  Price: ${price:.2f}")
print(f"  Size: {size:.2f} shares")
print(f"  Total: ${actual_cost:.2f}")

In [ ]:
# Confirm and place order
confirm = input("\nPlace order? (yes/no): ").strip().lower()

if confirm == "yes":
    try:
        order_args = OrderArgs(
            price=price,
            size=size,
            side=side,
            token_id=token_id,
        )
        
        signed_order = client.create_order(order_args)
        result = client.post_order(signed_order, OrderType.GTC)
        
        print("\n" + "=" * 50)
        print("ORDER PLACED SUCCESSFULLY!")
        print("=" * 50)
        print(f"Order ID: {result.get('orderID', 'N/A')}")
        print(f"Status: {result.get('status', 'N/A')}")
        
    except Exception as e:
        print(f"\n❌ Order failed: {e}")
else:
    print("\nOrder cancelled.")

## 5. Check Updated Balance & Positions

In [ ]:
# Refresh balance
balance_data = client.get_balance_allowance(balance_params)
balance_usdc = int(balance_data.get("balance", 0)) / 1_000_000

print(f"Updated Balance: ${balance_usdc:.2f} USDC")

# Check trades
trades = client.get_trades()
print(f"\nRecent Trades: {len(trades)}")
for trade in trades[:3]:
    print(f"  - {trade.get('side')} {trade.get('size')} @ ${trade.get('price')}")

## 6. Cancel Orders